# Experiment 1
## Benchmark Qwen-VL-2.7B against OKVQA Dataset

In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 117.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 225.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 221.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 159.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 278.9 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
pip install transformers==4.51.3 accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 113.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.9/798.9 kB 118.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 155.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
# It's highly recommended to use `[decord]` feature for faster video loading.
!pip install qwen-vl-utils[decord]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.7/39.7 MB 108.8 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 161.9 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [4]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"PyTorch CUDA version: {torch.version.cuda}")
print(f"Has uint64: {hasattr(torch, 'uint64')}")

PyTorch version: 2.8.0.dev20250319+cu128
CUDA available: True
PyTorch CUDA version: 12.8
Has uint64: True


In [5]:
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor
import torch
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2.5-VL-7B-Instruct",
    torch_dtype=torch.float16,
    device_map="auto"  # Automatically distributes across available GPUs
)

config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/216 [00:00<?, ?B/s]

In [6]:
processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-7B-Instruct")

preprocessor_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

chat_template.json: 0.00B [00:00, ?B/s]

In [7]:
!mkdir -p data/okvqa

In [8]:
!apt-get update && apt-get install -y unzip

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy InRelease [270 kB]                
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1581 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]        
Get:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]      
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1927 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3209 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [5111 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy/restricted amd64 Packages [164 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/multiverse amd64 Packages [48.5 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1270 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy/

In [9]:
!wget -c https://okvqa.allenai.org/static/data/mscoco_train2014_annotations.json.zip
!wget -c https://okvqa.allenai.org/static/data/OpenEnded_mscoco_train2014_questions.json.zip
!wget -c https://okvqa.allenai.org/static/data/mscoco_val2014_annotations.json.zip
!wget -c https://okvqa.allenai.org/static/data/OpenEnded_mscoco_val2014_questions.json.zip

--2025-08-12 15:32:43--  https://okvqa.allenai.org/static/data/mscoco_train2014_annotations.json.zip
Resolving okvqa.allenai.org (okvqa.allenai.org)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to okvqa.allenai.org (okvqa.allenai.org)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 711437 (695K) [application/x-zip-compressed]
Saving to: ‘mscoco_train2014_annotations.json.zip’

mscoco_train2014_an 100%[===================>] 694.76K  --.-KB/s    in 0.01s   

2025-08-12 15:32:44 (45.5 MB/s) - ‘mscoco_train2014_annotations.json.zip’ saved [711437/711437]

--2025-08-12 15:32:44--  https://okvqa.allenai.org/static/data/OpenEnded_mscoco_train2014_questions.json.zip
Resolving okvqa.allenai.org (okvqa.allenai.org)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to okvqa.allenai.org (okvqa.allenai.org)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 209933 (205K) [appl

In [10]:
!unzip -o mscoco_train2014_annotations.json.zip
!unzip -o OpenEnded_mscoco_train2014_questions.json.zip
!unzip -o mscoco_val2014_annotations.json.zip
!unzip -o OpenEnded_mscoco_val2014_questions.json.zip

Archive:  mscoco_train2014_annotations.json.zip
  inflating: mscoco_train2014_annotations.json  
Archive:  OpenEnded_mscoco_train2014_questions.json.zip
  inflating: OpenEnded_mscoco_train2014_questions.json  
   creating: __MACOSX/
  inflating: __MACOSX/._OpenEnded_mscoco_train2014_questions.json  
Archive:  mscoco_val2014_annotations.json.zip
  inflating: mscoco_val2014_annotations.json  
Archive:  OpenEnded_mscoco_val2014_questions.json.zip
  inflating: OpenEnded_mscoco_val2014_questions.json  
  inflating: __MACOSX/._OpenEnded_mscoco_val2014_questions.json  


In [7]:
import json

with open("data/okvqa/mscoco_train2014_annotations.json") as f:
    train_ann = json.load(f)
with open("data/okvqa/OpenEnded_mscoco_train2014_questions.json") as f:
    train_qs = json.load(f)
with open("data/okvqa/mscoco_val2014_annotations.json") as f:
    val_ann = json.load(f)
with open("data/okvqa/OpenEnded_mscoco_val2014_questions.json") as f:
    val_qs = json.load(f)

print("Train questions:", len(train_qs["questions"]))
print("Train annotations:", len(train_ann["annotations"]))
print("Val questions:", len(val_qs["questions"]))
print("Val annotations:", len(val_ann["annotations"]))

Train questions: 9009
Train annotations: 9009
Val questions: 5046
Val annotations: 5046


In [8]:
!pip install matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 159.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 251.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 201.2 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [9]:
!pip install tqdm


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [11]:
# OK-VQA Accuracy Scoring:
# Each question in OK-VQA has 10 human-provided ground truth answers.
# To evaluate a predicted answer:
# 1. Normalize the predicted answer (lowercase, remove punctuation and articles).
# 2. Normalize all 10 ground truth answers similarly.
# 3. Count how many ground truth answers exactly match the predicted answer (call this count N).
# 4. Compute accuracy as min(N / 3, 1.0):
#    - If 3 or more annotators agreed, accuracy = 1.0 (full credit).
#    - If fewer than 3 annotators agreed, partial credit = N/3.
#    - If no matches, accuracy = 0.
# This scoring reflects agreement level between the prediction and annotators.

In [10]:
import json
import requests
from PIL import Image
from io import BytesIO
import torch
from tqdm import tqdm
import re
from collections import Counter
import time
import matplotlib.pyplot as plt
import string
from IPython.display import display, clear_output
import itertools

def processPunctuation(inText):
    """
    Official VQA preprocessing for punctuation - CORRECTED VERSION
    
    From official spec:
    - Replace all punctuation (except apostrophe and colon) with a space character
    - Do not remove apostrophe (') or colon (:)  
    - For commas between digits, remove comma entirely (no space)
    """
    outText = inText
    
    # Step 1: Handle commas between digits first (remove entirely, no space)
    outText = re.sub(r'(\d),(\d)', r'\1\2', outText)
    
    # Step 2: Replace all other punctuation (except ' and :) with spaces
    # Get all punctuation except apostrophe and colon
    punctuation_to_replace = set(string.punctuation) - {"'", ":"}
    
    for p in punctuation_to_replace:
        if p in outText:
            outText = outText.replace(p, ' ')
    
    # Step 3: Clean up multiple spaces
    outText = re.sub(r'\s+', ' ', outText).strip()
    
    return outText

def processDigitArticle(inText):
    """Official VQA preprocessing for digits and articles"""
    outText = []
    tempText = inText.lower().split()
    articles = ['a', 'an', 'the']
    manualMap = { 'none': '0',
                  'zero': '0',
                  'one': '1',
                  'two': '2',
                  'three': '3',
                  'four': '4',
                  'five': '5',
                  'six': '6',
                  'seven': '7',
                  'eight': '8',
                  'nine': '9',
                  'ten': '10'}
    contractions = {"aint": "ain't", "arent": "aren't", "cant": "can't", "couldve": "could've", 
                    "couldnt": "couldn't", "couldn'tve": "couldn't've", "couldnt've": "couldn't've", 
                    "didnt": "didn't", "doesnt": "doesn't", "dont": "don't", "hadnt": "hadn't", 
                    "hadnt've": "hadn't've", "hadn'tve": "hadn't've", "hasnt": "hasn't", "havent": "haven't", 
                    "hed": "he'd", "hed've": "he'd've", "he'dve": "he'd've", "hes": "he's", "howd": "how'd", 
                    "howll": "how'll", "hows": "how's", "Id": "I'd", "Id've": "I'd've", "I'dve": "I'd've", 
                    "Im": "I'm", "Ive": "I've", "isnt": "isn't", "itd": "it'd", "itd've": "it'd've", 
                    "it'dve": "it'd've", "itll": "it'll", "let's": "let's", "maam": "ma'am", "mightnt": "mightn't", 
                    "mightnt've": "mightn't've", "mightn'tve": "mightn't've", "mightve": "might've", 
                    "mustnt": "mustn't", "mustve": "must've", "neednt": "needn't", "notve": "not've", 
                    "oclock": "o'clock", "oughtnt": "oughtn't", "ow's'at": "'ow's'at", "'ows'at": "'ow's'at", 
                    "'ow'sat": "'ow's'at", "shant": "shan't", "shed've": "she'd've", "she'dve": "she'd've", 
                    "she's": "she's", "shouldve": "should've", "shouldnt": "shouldn't", "shouldnt've": "shouldn't've", 
                    "shouldn'tve": "shouldn't've", "somebody'd": "somebodyd", "somebodyd've": "somebody'd've", 
                    "somebody'dve": "somebody'd've", "somebodyll": "somebody'll", "somebodys": "somebody's", 
                    "someoned": "someone'd", "someoned've": "someone'd've", "someone'dve": "someone'd've", 
                    "someonell": "someone'll", "someones": "someone's", "somethingd": "something'd", 
                    "somethingd've": "something'd've", "something'dve": "something'd've", "somethingll": "something'll", 
                    "thats": "that's", "thered": "there'd", "thered've": "there'd've", "there'dve": "there'd've", 
                    "therere": "there're", "theres": "there's", "theyd": "they'd", "theyd've": "they'd've", 
                    "they'dve": "they'd've", "theyll": "they'll", "theyre": "they're", "theyve": "they've", 
                    "twas": "'twas", "wasnt": "wasn't", "wed've": "we'd've", "we'dve": "we'd've", "weve": "we've", 
                    "werent": "weren't", "whatll": "what'll", "whatre": "what're", "whats": "what's", 
                    "whatve": "what've", "whered": "where'd", "wheres": "where's", "whereve": "where've", 
                    "whod": "who'd", "whod've": "who'd've", "who'dve": "who'd've", "wholl": "who'll", 
                    "whos": "who's", "whove": "who've", "whyll": "why'll", "whyre": "why're", "whys": "why's", 
                    "wont": "won't", "wouldve": "would've", "wouldnt": "wouldn't", "wouldnt've": "wouldn't've", 
                    "wouldn'tve": "wouldn't've", "yall": "y'all", "yall'll": "y'all'll", "y'allll": "y'all'll", 
                    "yall'd've": "y'all'd've", "y'alld've": "y'all'd've", "y'all'dve": "y'all'd've", 
                    "youd": "you'd", "youd've": "you'd've", "you'dve": "you'd've", "youll": "you'll", 
                    "youre": "you're", "youve": "you've"}
    
    for word in tempText:
        word = manualMap.setdefault(word, word)
        if word not in articles:
            outText.append(word)
    for wordId, word in enumerate(outText):
        if word in contractions:
            outText[wordId] = contractions[word]
    outText = ' '.join(outText)
    return outText

def preprocess_answer_fixed(answer):
    """
    Complete VQA preprocessing pipeline - FIXED VERSION
    """
    if not answer:
        return ""
    
    answer = str(answer)
    
    # Step 1: Lowercase
    answer = answer.lower()
    
    # Step 2: Remove periods except decimals
    answer = re.sub('(?<!\\d)\\.(?!\\d)', '', answer)
    
    # Step 3: Process digits and articles (existing function is correct)
    answer = processDigitArticle(answer)
    
    # Step 4: Process punctuation (FIXED VERSION)
    answer = processPunctuation(answer)
    
    # Step 5: Final cleanup
    answer = ' '.join(answer.strip().split())
    
    return answer


def compute_vqa_accuracy_okvqa(prediction, ground_truth_answers):
    """
    Compute OK-VQA style accuracy: average VQA score over all 10 subsets of 9 human answers.
    """
    if not prediction or not ground_truth_answers:
        return 0.0

    # Preprocess the prediction
    proc_pred = preprocess_answer_fixed(prediction)

    # Extract and preprocess GT answers (supporting dict format or raw strings)
    raw_gt = [ans["answer"] if isinstance(ans, dict) else str(ans) for ans in ground_truth_answers]
    proc_gt = [preprocess_answer_fixed(ans) for ans in raw_gt]

    if len(proc_gt) != 10:
        # If there aren't exactly 10 GT answers, fall back to standard VQA style
        matches = proc_gt.count(proc_pred)
        return min(matches, 3) / 3.0

    scores = []
    # Loop over all subsets of 9 answers by dropping one each time
    for dropped_idx in range(10):
        subset = [ans for i, ans in enumerate(proc_gt) if i != dropped_idx]
        matches = subset.count(proc_pred)
        score = min(matches / 3.0, 1.0)
        scores.append(score)

    # Return average across all 10 subsets
    return sum(scores) / len(scores)


def compute_vqa_accuracy_debug(prediction, ground_truth_answers):
    """Debug version showing the official VQA preprocessing steps"""
    if not prediction or not ground_truth_answers:
        return 0.0
    
    print(f"\n--- Official VQA Accuracy Debug ---")
    print(f"Raw Prediction: '{prediction}'")
    
    # Show preprocessing steps
    processed_pred = preprocess_answer_fixed(prediction)
    print(f"Processed Prediction: '{processed_pred}'")
    
    # Handle different GT answer formats
    if isinstance(ground_truth_answers[0], dict):
        raw_gt = [ans["answer"] for ans in ground_truth_answers]
        processed_gt = [preprocess_answer_fixed(ans["answer"]) for ans in ground_truth_answers]
    else:
        raw_gt = [str(ans) for ans in ground_truth_answers]
        processed_gt = [preprocess_answer_fixed(str(ans)) for ans in ground_truth_answers]
    
    print(f"Raw GT Answers: {raw_gt}")
    print(f"Processed GT Answers: {processed_gt}")
    
    # Count exact matches
    exact_matches = processed_gt.count(processed_pred)
    print(f"Exact matches found: {exact_matches}")
    
    # Show which ones matched
    for i, gt in enumerate(processed_gt):
        match_status = "✓" if gt == processed_pred else "✗"
        print(f"  {match_status} '{processed_pred}' vs '{gt}'")
    
    # Apply official VQA scoring
    vqa_score = min(exact_matches, 3) / 3.0
    print(f"VQA Score: min({exact_matches}, 3) / 3 = {vqa_score}")
    
    return vqa_score

def extract_short_answer(full_response):
    """Extract short answer from model's full response"""
    if not full_response:
        return ""
    
    response = full_response.strip()
    # Remove markdown formatting
    response = re.sub(r'\*\*([^*]+)\*\*', r'\1', response)
    response = re.sub(r'\*([^*]+)\*', r'\1', response)
    
    # Patterns to extract answers
    patterns = [
        r'appears to be (?:a|an|the)?\s*([^,\.\(\)]+)',
        r'is (?:a|an|the)?\s*([^,\.\(\)]+)',
        r'looks like (?:a|an|the)?\s*([^,\.\(\)]+)',
        r'seems to be (?:a|an|the)?\s*([^,\.\(\)]+)',
        r'called (?:a|an|the)?\s*([^,\.\(\)]+)',
        r'known as (?:a|an|the)?\s*([^,\.\(\)]+)',
        r'^(?:The|This|It|That)?\s*(?:is|appears|looks)\s*(?:a|an|the)?\s*([^,\.\(\)]+)',
        r'"([^"]+)"',
        r'its?\s*"?([^",\.\(\)]+)"?',
    ]
    
    for pattern in patterns:
        match = re.search(pattern, response, re.IGNORECASE)
        if match:
            answer = match.group(1).strip()
            answer = re.sub(r'\s+', ' ', answer)
            if len(answer) > 2 and len(answer.split()) <= 4:
                return answer
    
    # Fallback: extract meaningful words
    words = response.split()
    if words:
        skip_words = {'the', 'this', 'it', 'that', 'appears', 'seems', 'looks', 'is', 'a', 'an'}
        meaningful_words = []
        for word in words:
            clean_word = re.sub(r'[^\w]', '', word.lower())
            if clean_word not in skip_words and len(clean_word) > 1:
                meaningful_words.append(word)
                if len(meaningful_words) >= 2:
                    break
        if meaningful_words:
            return ' '.join(meaningful_words[:2])
    
    return ' '.join(response.split()[:3])

def generate_answer_qwen(image, question, model, processor):
    """Generate answer using Qwen model"""
    try:
        prompts = [
            f"Question: {question}\nProvide a very short answer (1-2 words):",
            f"Answer this question with just 1-2 words: {question}",
            f"{question}\nShort answer:",
        ]
        
        best_answer = ""
        for prompt in prompts:
            messages = [
                {
                    "role": "user",
                    "content": [
                        {"type": "image", "image": image},
                        {"type": "text", "text": prompt}
                    ]
                }
            ]
            
            text = processor.apply_chat_template(
                messages, tokenize=False, add_generation_prompt=True
            )
            inputs = processor(
                text=text,
                images=image,
                return_tensors="pt"
            )
            inputs = inputs.to(model.device)
            
            with torch.no_grad():
                outputs = model.generate(
                    **inputs,
                    max_new_tokens=25,
                    do_sample=False,
                    pad_token_id=processor.tokenizer.eos_token_id
                )
            
            input_length = inputs['input_ids'].shape[1]
            generated_tokens = outputs[0][input_length:]
            response = processor.tokenizer.decode(generated_tokens, skip_special_tokens=True)
            
            short_answer = extract_short_answer(response)
            if short_answer and (not best_answer or len(short_answer) < len(best_answer)):
                best_answer = short_answer
            
            if short_answer and len(short_answer.split()) <= 2:
                break
                
        return best_answer.strip()
    except Exception as e:
        print(f"Error generating answer: {e}")
        return ""

def download_image_with_retry(image_id, max_retries=3):
    """Download image from COCO dataset with retry logic"""
    url = f"http://images.cocodataset.org/val2014/COCO_val2014_{image_id:012d}.jpg"
    for attempt in range(max_retries):
        try:
            response = requests.get(url, timeout=15)
            response.raise_for_status()
            image = Image.open(BytesIO(response.content)).convert('RGB')
            return image
        except Exception as e:
            if attempt == max_retries - 1:
                print(f"Failed to download image {image_id}: {e}")
                return None
            time.sleep(1)
    return None

def run_okvqa_benchmark(model, processor, num_samples=None, save_results=True):
    """Run OK-VQA benchmark evaluation with official VQA accuracy metric"""
    print("Loading OK-VQA data...")
    try:
        with open("data/okvqa/OpenEnded_mscoco_val2014_questions.json") as f:
            questions_data = json.load(f)
        with open("data/okvqa/mscoco_val2014_annotations.json") as f:
            annotations_data = json.load(f)
    except FileNotFoundError as e:
        print(f"Data file not found: {e}")
        print("Make sure you've run the setup script to download OK-VQA data")
        return 0.0, []

    questions = questions_data['questions']
    annotations = {ann["question_id"]: ann for ann in annotations_data['annotations']}

    if num_samples is not None:
        questions = questions[:num_samples]
        print(f"Evaluating on {num_samples} samples")
    else:
        print(f"Evaluating on full validation set ({len(questions)} samples)")

    correct = 0.0
    total = 0
    failed_downloads = 0
    processing_errors = 0
    results = []

    print("Starting evaluation with official VQA metric...")

    for i, question_data in enumerate(questions):
        try:
            question_id = question_data["question_id"]
            image_id = question_data["image_id"]
            question = question_data["question"]

            if question_id not in annotations:
                continue

            annotation = annotations[question_id]
            gt_answers = annotation["answers"]

            image = download_image_with_retry(image_id)
            if image is None:
                failed_downloads += 1
                continue

            prediction = generate_answer_qwen(image, question, model, processor)
            if not prediction:
                processing_errors += 1
                continue

            # Use official VQA accuracy evaluation
            accuracy = compute_vqa_accuracy_okvqa(prediction, gt_answers)
            correct += accuracy
            total += 1

            results.append({
                "question_id": question_id,
                "image_id": image_id,
                "question": question,
                "prediction": prediction,
                "ground_truth": [ans["answer"] for ans in gt_answers],
                "vqa_accuracy": accuracy,
                "exact_match": accuracy == 1.0
            })

            # Stream display
            if plt.get_fignums():  # Clear previous plot if exists
                plt.clf()
            
            plt.figure(figsize=(10, 6))
            plt.imshow(image)
            plt.axis('off')
            plt.title(f"Q: {question}\nPredicted: {prediction}\nAccuracy: {accuracy:.2f}", 
                     fontsize=10, wrap=True)
            
            if hasattr(plt, 'tight_layout'):
                plt.tight_layout()
            
            try:
                display(plt.gcf())
                plt.close()
                clear_output(wait=True)
            except:
                pass  # Skip display if not in notebook
            
            time.sleep(0.1)  # Brief pause

            if (i + 1) % 25 == 0:
                print(f"\nProgress: {i+1}/{len(questions)}")
                print(f"Current VQA Accuracy: {correct / total:.4f}")
                print(f"Processed: {total}, Failed downloads: {failed_downloads}, Processing errors: {processing_errors}")
                print(f"Last Q: {question}")
                print(f"Last A: {prediction} | GT: {[a['answer'] for a in gt_answers[:3]]}")

        except Exception as e:
            print(f"Error processing question {question_id}: {e}")
            processing_errors += 1
            continue

    final_accuracy = correct / total if total > 0 else 0
    exact_match_count = sum(1 for r in results if r["exact_match"])
    exact_match_rate = exact_match_count / total if total > 0 else 0

    print("\n" + "="*70)
    print("FINAL BENCHMARK RESULTS (Official VQA Metric)")
    print("="*70)
    print(f"Model: Qwen/Qwen2.5-VL-7B-Instruct")
    print(f"Dataset: OK-VQA Validation Set")
    print(f"Total questions processed: {total}")
    print(f"Failed image downloads: {failed_downloads}")
    print(f"Processing errors: {processing_errors}")
    print(f"")
    print(f"VQA Accuracy: {final_accuracy:.4f} ({final_accuracy*100:.2f}%)")
    print(f"Exact Match Rate: {exact_match_rate:.4f} ({exact_match_rate*100:.2f}%)")
    print(f"Correct answers (weighted): {correct:.1f}/{total}")
    print(f"Exact matches: {exact_match_count}/{total}")

    print(f"\n--- Sample Results ---")
    for i, result in enumerate(results[:5]):
        status = "✓" if result["vqa_accuracy"] > 0 else "✗"
        print(f"{status} Q: {result['question']}")
        print(f"   Pred: '{result['prediction']}'")
        print(f"   GT: {result['ground_truth']}")
        print(f"   Score: {result['vqa_accuracy']:.2f}")
        print()

    if save_results and results:
        filename = f"okvqa_qwen2.5vl_results_{total}samples.json"
        summary = {
            "model_name": "Qwen/Qwen2.5-VL-7B-Instruct",
            "dataset": "OK-VQA validation",
            "evaluation_date": time.strftime("%Y-%m-%d %H:%M:%S"),
            "evaluation_method": "Official VQA Accuracy Metric",
            "total_processed": total,
            "failed_downloads": failed_downloads,
            "processing_errors": processing_errors,
            "vqa_accuracy": final_accuracy,
            "exact_match_rate": exact_match_rate,
            "weighted_correct": correct,
            "exact_matches": exact_match_count,
            "sample_results": results
        }
        with open(filename, 'w') as f:
            json.dump(summary, f, indent=2)
        print(f"Detailed results saved to: {filename}")

    return final_accuracy, results

def analyze_errors(results):
    """Analyze errors in VQA results"""
    if not results:
        return

    print("\n=== ERROR ANALYSIS ===")

    correct = [r for r in results if r["vqa_accuracy"] > 0]
    incorrect = [r for r in results if r["vqa_accuracy"] == 0]

    print(f"Correct predictions: {len(correct)} ({len(correct)/len(results)*100:.1f}%)")
    print(f"Incorrect predictions: {len(incorrect)} ({len(incorrect)/len(results)*100:.1f}%)")

    print(f"\n--- Error Examples ---")
    for i, result in enumerate(incorrect[:5]):
        print(f"Q: {result['question']}")
        print(f"Pred: '{result['prediction']}'")
        print(f"GT: {result['ground_truth']}")
        print()

# Usage examples
if __name__ == "__main__":
    print("Usage examples:")
    print("1. Run full OK-VQA benchmark:")
    print("   accuracy, results = run_okvqa_benchmark(model, processor)")
    print()
    print("2. Run on subset for testing:")
    print("   accuracy, results = run_okvqa_benchmark(model, processor, num_samples=10)")
    print()
    print("3. Debug a specific prediction with official VQA preprocessing:")
    print("   score = compute_vqa_accuracy_debug('Shopping', ['shop']*6 + ['nyc']*2 + ['business']*2)")
    print()

Usage examples:
1. Run full OK-VQA benchmark:
   accuracy, results = run_okvqa_benchmark(model, processor)

2. Run on subset for testing:
   accuracy, results = run_okvqa_benchmark(model, processor, num_samples=10)

3. Debug a specific prediction with official VQA preprocessing:
   score = compute_vqa_accuracy_debug('Shopping', ['shop']*6 + ['nyc']*2 + ['business']*2)



In [17]:
run_okvqa_benchmark(model, processor)


FINAL BENCHMARK RESULTS (Official VQA Metric)
Model: Qwen/Qwen2.5-VL-7B-Instruct
Dataset: OK-VQA Validation Set
Total questions processed: 5046
Failed image downloads: 0
Processing errors: 0

VQA Accuracy: 0.5501 (55.01%)
Exact Match Rate: 0.4746 (47.46%)
Correct answers (weighted): 2776.0/5046
Exact matches: 2395/5046

--- Sample Results ---
✗ Q: What sport can you use this for?
   Pred: 'motorcycle racing'
   GT: ['race', 'race', 'race', 'race', 'race', 'race', 'motocross', 'motocross', 'ride', 'ride']
   Score: 0.00

✗ Q: Name the type of plant this is?
   Pred: 'pothos'
   GT: ['vine', 'vine', 'vine', 'vine', 'climb', 'climb', 'look like some kind of ivy', 'look like some kind of ivy', 'ficus', 'ficus']
   Score: 0.00

✓ Q: What toy is this?
   Pred: 'teddy bear'
   GT: ['stuffed animal', 'stuffed animal', 'stuffed animal', 'stuffed animal', 'stuffed animal', 'stuffed animal', 'teddy bear', 'teddy bear', 'teddy bear', 'teddy bear']
   Score: 1.00

✓ Q: Which part of this animal wo

(0.5501387237415717,
 [{'question_id': 2971475,
   'image_id': 297147,
   'question': 'What sport can you use this for?',
   'prediction': 'motorcycle racing',
   'ground_truth': ['race',
    'race',
    'race',
    'race',
    'race',
    'race',
    'motocross',
    'motocross',
    'ride',
    'ride'],
   'vqa_accuracy': 0.0,
   'exact_match': False},
  {'question_id': 3397615,
   'image_id': 339761,
   'question': 'Name the type of plant this is?',
   'prediction': 'pothos',
   'ground_truth': ['vine',
    'vine',
    'vine',
    'vine',
    'climb',
    'climb',
    'look like some kind of ivy',
    'look like some kind of ivy',
    'ficus',
    'ficus'],
   'vqa_accuracy': 0.0,
   'exact_match': False},
  {'question_id': 3575865,
   'image_id': 357586,
   'question': 'What toy is this?',
   'prediction': 'teddy bear',
   'ground_truth': ['stuffed animal',
    'stuffed animal',
    'stuffed animal',
    'stuffed animal',
    'stuffed animal',
    'stuffed animal',
    'teddy bear'

In [18]:
run_okvqa_benchmark(model, processor, num_samples=10)


FINAL BENCHMARK RESULTS (Official VQA Metric)
Model: Qwen/Qwen2.5-VL-7B-Instruct
Dataset: OK-VQA Validation Set
Total questions processed: 10
Failed image downloads: 0
Processing errors: 0

VQA Accuracy: 0.2600 (26.00%)
Exact Match Rate: 0.2000 (20.00%)
Correct answers (weighted): 2.6/10
Exact matches: 2/10

--- Sample Results ---
✗ Q: What sport can you use this for?
   Pred: 'motorcycle racing'
   GT: ['race', 'race', 'race', 'race', 'race', 'race', 'motocross', 'motocross', 'ride', 'ride']
   Score: 0.00

✗ Q: Name the type of plant this is?
   Pred: 'pothos'
   GT: ['vine', 'vine', 'vine', 'vine', 'climb', 'climb', 'look like some kind of ivy', 'look like some kind of ivy', 'ficus', 'ficus']
   Score: 0.00

✓ Q: What toy is this?
   Pred: 'teddy bear'
   GT: ['stuffed animal', 'stuffed animal', 'stuffed animal', 'stuffed animal', 'stuffed animal', 'stuffed animal', 'teddy bear', 'teddy bear', 'teddy bear', 'teddy bear']
   Score: 1.00

✓ Q: Which part of this animal would be in us

(0.26,
 [{'question_id': 2971475,
   'image_id': 297147,
   'question': 'What sport can you use this for?',
   'prediction': 'motorcycle racing',
   'ground_truth': ['race',
    'race',
    'race',
    'race',
    'race',
    'race',
    'motocross',
    'motocross',
    'ride',
    'ride'],
   'vqa_accuracy': 0.0,
   'exact_match': False},
  {'question_id': 3397615,
   'image_id': 339761,
   'question': 'Name the type of plant this is?',
   'prediction': 'pothos',
   'ground_truth': ['vine',
    'vine',
    'vine',
    'vine',
    'climb',
    'climb',
    'look like some kind of ivy',
    'look like some kind of ivy',
    'ficus',
    'ficus'],
   'vqa_accuracy': 0.0,
   'exact_match': False},
  {'question_id': 3575865,
   'image_id': 357586,
   'question': 'What toy is this?',
   'prediction': 'teddy bear',
   'ground_truth': ['stuffed animal',
    'stuffed animal',
    'stuffed animal',
    'stuffed animal',
    'stuffed animal',
    'stuffed animal',
    'teddy bear',
    'teddy b

In [19]:
run_okvqa_benchmark(model, processor, num_samples=100)


Progress: 100/100
Current VQA Accuracy: 0.5120
Processed: 100, Failed downloads: 0, Processing errors: 0
Last Q: How much tomato sauce is in this dish?
Last A: lot | GT: ['2 cups', '2 cups', '2 cups']

FINAL BENCHMARK RESULTS (Official VQA Metric)
Model: Qwen/Qwen2.5-VL-7B-Instruct
Dataset: OK-VQA Validation Set
Total questions processed: 100
Failed image downloads: 0
Processing errors: 0

VQA Accuracy: 0.5120 (51.20%)
Exact Match Rate: 0.4100 (41.00%)
Correct answers (weighted): 51.2/100
Exact matches: 41/100

--- Sample Results ---
✗ Q: What sport can you use this for?
   Pred: 'motorcycle racing'
   GT: ['race', 'race', 'race', 'race', 'race', 'race', 'motocross', 'motocross', 'ride', 'ride']
   Score: 0.00

✗ Q: Name the type of plant this is?
   Pred: 'pothos'
   GT: ['vine', 'vine', 'vine', 'vine', 'climb', 'climb', 'look like some kind of ivy', 'look like some kind of ivy', 'ficus', 'ficus']
   Score: 0.00

✓ Q: What toy is this?
   Pred: 'teddy bear'
   GT: ['stuffed animal', 

(0.5120000000000001,
 [{'question_id': 2971475,
   'image_id': 297147,
   'question': 'What sport can you use this for?',
   'prediction': 'motorcycle racing',
   'ground_truth': ['race',
    'race',
    'race',
    'race',
    'race',
    'race',
    'motocross',
    'motocross',
    'ride',
    'ride'],
   'vqa_accuracy': 0.0,
   'exact_match': False},
  {'question_id': 3397615,
   'image_id': 339761,
   'question': 'Name the type of plant this is?',
   'prediction': 'pothos',
   'ground_truth': ['vine',
    'vine',
    'vine',
    'vine',
    'climb',
    'climb',
    'look like some kind of ivy',
    'look like some kind of ivy',
    'ficus',
    'ficus'],
   'vqa_accuracy': 0.0,
   'exact_match': False},
  {'question_id': 3575865,
   'image_id': 357586,
   'question': 'What toy is this?',
   'prediction': 'teddy bear',
   'ground_truth': ['stuffed animal',
    'stuffed animal',
    'stuffed animal',
    'stuffed animal',
    'stuffed animal',
    'stuffed animal',
    'teddy bear'

# Experiment 2
## Generating Captions for the images in the test dataset to see the features that the model weighs more (text or visual) to generate the answers and compare the results against the Experiment 1 Results

In [11]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 786.8/786.8 kB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 118.2 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [12]:
import openai
import os
import base64
from io import BytesIO

In [13]:
# Load the API key from the environment variable
openai.api_key = os.getenv("OPENAI_API_KEY")

# You can add a check to ensure the key was loaded
if not openai.api_key:
    print("OpenAI API key not found. Please set it as an environment variable in your pod settings.")
else:
    print("OpenAI API key loaded successfully.")

OpenAI API key loaded successfully.


In [17]:
def generate_captions(image):
    """
    Generates a short, descriptive caption for an image using the OpenAI GPT-4o model.
    """
    if image is None:
        return "No image provided for captioning."

    try:
        # Convert the PIL Image to a base64 string
        buffered = BytesIO()
        image.save(buffered, format="JPEG")
        img_base64 = base64.b64encode(buffered.getvalue()).decode("utf-8")

        # Make the API call to OpenAI
        response = openai.chat.completions.create(
            model="gpt-4o",
            messages=[
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": "Generate a concise, one-sentence descriptive caption for this image."},
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{img_base64}"
                            }
                        }
                    ]
                }
            ],
            max_tokens=60  # Limit the caption length
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error generating OpenAI caption: {e}")
        return "Error: Caption generation failed."

In [22]:
import json
from tqdm import tqdm
import time
import os

def generate_captions_val_set(num_samples=None):
    """
    Generates a caption for EACH question entry in the OK-VQA validation set
    and saves them to a file mapping question_id to caption.
    """
    print("Loading OK-VQA questions...")
    try:
        with open("data/okvqa/OpenEnded_mscoco_val2014_questions.json") as f:
            questions_data = json.load(f)
    except FileNotFoundError as e:
        print(f"Question file not found: {e}")
        return

    questions = questions_data['questions']
    if num_samples:
        questions = questions[:num_samples]

    print(f"Preparing to generate captions for all {len(questions)} questions.")

    captions = {}
    output_filename = f"okvqa_openai_captions_{len(questions)}questions.json"
    
    # Check if a captions file exists to resume from
    if os.path.exists(output_filename):
        with open(output_filename, 'r') as f:
            captions = json.load(f)
        print(f"Resuming... Found {len(captions)} captions already generated in {output_filename}.")

    # --- Main Loop: Iterate through every question ---
    for question_data in tqdm(questions, desc="Generating Captions for Each Question"):
        question_id = str(question_data["question_id"]) # Use string for JSON keys
        image_id = question_data["image_id"]

        # Skip if this question_id has already been processed
        if question_id in captions:
            continue

        # Download the image associated with the current question
        image = download_image_with_retry(image_id)
        if image:
            # Generate a new caption
            caption = generate_captions(image)
            captions[question_id] = caption
        else:
            captions[question_id] = "Image download failed"

        # Save progress every 25 captions to prevent data loss
        if len(captions) % 25 == 0:
            with open(output_filename, 'w') as f:
                json.dump(captions, f, indent=2)

    # Final save at the end of the process
    with open(output_filename, 'w') as f:
        json.dump(captions, f, indent=2)

    print(f"\nCaption generation complete. {len(captions)} captions saved to {output_filename}")
    return captions

In [24]:
# This will generate and save captions for all images.
all_captions = generate_captions_val_set() 

Loading OK-VQA questions...
Preparing to generate captions for all 5046 questions.


Generating Captions for Each Question: 100%|██████████| 5046/5046 [2:32:38<00:00,  1.81s/it]  


Caption generation complete. 5046 captions saved to okvqa_openai_captions_5046questions.json


In [26]:
import json

# Define the filename you want to inspect
filename = "okvqa_qwen2.5vl_results_5046samples.json"

try:
    # Open and load the JSON file
    with open(filename, 'r') as f:
        data = json.load(f)

    # Check if the 'sample_results' key exists and get its length
    if 'sample_results' in data:
        num_entries = len(data['sample_results'])
        print(f"The file '{filename}' contains {num_entries} entries in the 'sample_results' list.")
    else:
        print(f"Error: The key 'sample_results' was not found in '{filename}'.")
        
    # As a cross-reference, you can also print the 'total_processed' field
    if 'total_processed' in data:
        total_processed = data['total_processed']
        print(f"The 'total_processed' field in the file reports {total_processed} items.")

except FileNotFoundError:
    print(f"Error: The file '{filename}' was not found. Please make sure the file name is correct and it is in the same directory as the notebook.")
except json.JSONDecodeError:
    print(f"Error: The file '{filename}' is not a valid JSON file and could not be read.")

The file 'okvqa_qwen2.5vl_results_5046samples.json' contains 5046 entries in the 'sample_results' list.
The 'total_processed' field in the file reports 5046 items.


In [27]:
def run_benchmark_with_pregenerated_captions(model, processor, captions_filepath, num_samples=None, save_results=True):
    """
    Run OK-VQA benchmark using pre-generated captions mapped by question_id.
    """
    print("Loading OK-VQA data and pre-generated captions...")
    try:
        with open(captions_filepath, 'r') as f:
            captions_data = json.load(f)
        with open("data/okvqa/OpenEnded_mscoco_val2014_questions.json") as f:
            questions_data = json.load(f)
        with open("data/okvqa/mscoco_val2014_annotations.json") as f:
            annotations_data = json.load(f)
    except FileNotFoundError as e:
        print(f"Data file not found: {e}")
        return 0.0, []

    questions = questions_data['questions']
    annotations = {ann["question_id"]: ann for ann in annotations_data['annotations']}

    if num_samples is not None:
        questions = questions[:num_samples]

    print(f"Evaluating on {len(questions)} samples.")
    
    results = []
    correct = 0.0
    total = 0

    for question_data in tqdm(questions, desc="Running Benchmark"):
        image_id = question_data["image_id"]
        question_id = str(question_data["question_id"]) # Use string for JSON keys
        
        # *** KEY CHANGE: Look up the caption using question_id instead of image_id ***
        openai_caption = captions_data.get(question_id, "Caption not found.")
        
        gt_answers = annotations.get(int(question_id), {}).get("answers", [])
        if not gt_answers:
            continue

        image = download_image_with_retry(image_id)
        prediction = generate_answer_qwen(image, question_data["question"], model, processor)
        
        accuracy = compute_vqa_accuracy_okvqa(prediction, gt_answers)
        correct += accuracy
        total += 1
        
        results.append({
            "question_id": int(question_id),
            "image_id": image_id,
            "question": question_data["question"],
            "prediction": prediction,
            "openai_caption": openai_caption,
            "ground_truth": [ans["answer"] for ans in gt_answers],
            "vqa_accuracy": accuracy
        })

    final_accuracy = correct / total if total > 0 else 0
    print(f"\nFINAL VQA ACCURACY: {final_accuracy:.4f}")

    if save_results:
        filename = f"okvqa_benchmark_results_with_captions_{total}samples.json"
        with open(filename, 'w') as f:
            json.dump({"vqa_accuracy": final_accuracy, "results": results}, f, indent=2)
        print(f"Benchmark results saved to {filename}")

    return final_accuracy, results

In [28]:
# Define the path to the new captions file
captions_file = "okvqa_openai_captions_5046questions.json"

# Run the benchmark
run_benchmark_with_pregenerated_captions(model, processor, captions_file)

Loading OK-VQA data and pre-generated captions...
Evaluating on 5046 samples.


Running Benchmark:   0%|          | 0/5046 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `1e-06` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Running Benchmark: 100%|██████████| 5046/5046 [29:13<00:00,  2.88it/s]  


FINAL VQA ACCURACY: 0.5501
Benchmark results saved to okvqa_benchmark_results_with_captions_5046samples.json


(0.5501387237415717,
 [{'question_id': 2971475,
   'image_id': 297147,
   'question': 'What sport can you use this for?',
   'prediction': 'motorcycle racing',
   'openai_caption': 'A sleek black motorcycle is parked on a sunny day beside a metal structure.',
   'ground_truth': ['race',
    'race',
    'race',
    'race',
    'race',
    'race',
    'motocross',
    'motocross',
    'ride',
    'ride'],
   'vqa_accuracy': 0.0},
  {'question_id': 3397615,
   'image_id': 339761,
   'question': 'Name the type of plant this is?',
   'prediction': 'pothos',
   'openai_caption': 'A modern bathroom features a lush vertical garden, adding a touch of nature to the sleek, contemporary design.',
   'ground_truth': ['vine',
    'vine',
    'vine',
    'vine',
    'climb',
    'climb',
    'look like some kind of ivy',
    'look like some kind of ivy',
    'ficus',
    'ficus'],
   'vqa_accuracy': 0.0},
  {'question_id': 3575865,
   'image_id': 357586,
   'question': 'What toy is this?',
   'predic